## Combine JSON files and build tables

In [1]:
import os
import pandas as pd
from pandas.io.json import json_normalize
import seaborn as sns

In [2]:
datadir = '/home/freddy/insight/data/runbritain_scraped/v2/'

In [3]:
datadirs = []
for i,j,k in os.walk(datadir):
    tempstr = str(i)
    if 'events_' in tempstr:
        datadirs.append(i)

In [4]:
alljsonfiles = {}
njson = 0
for d in datadirs:
    jsonfiles = []
    for i,j,k in os.walk(d):
        jsonfiles.extend(k)
        break
    alljsonfiles[d] = jsonfiles
    njson += len(jsonfiles)

In [5]:
print('From %d directories, we found %d JSON files for collection.' 
      % (len(datadirs), njson))
for i in datadirs:
    print(i)

From 11 directories, we found 12014 JSON files for collection.
/home/freddy/insight/runbritain_scraped/v2/events_5
/home/freddy/insight/runbritain_scraped/v2/events_8
/home/freddy/insight/runbritain_scraped/v2/events_3
/home/freddy/insight/runbritain_scraped/v2/events_7
/home/freddy/insight/runbritain_scraped/v2/events_1
/home/freddy/insight/runbritain_scraped/v2/events_4
/home/freddy/insight/runbritain_scraped/v2/events_6
/home/freddy/insight/runbritain_scraped/v2/events_10
/home/freddy/insight/runbritain_scraped/v2/events_9
/home/freddy/insight/runbritain_scraped/v2/events_2
/home/freddy/insight/runbritain_scraped/v2/events_11


In [6]:
count = 0
dfs = []
for key, val in alljsonfiles.items():
    count = count + 1
    for idx, val2 in enumerate(val):
        filename = str(key) + '/' + str(val2)
        #print(filename)
        df = pd.read_json(filename)
        dfs.append(df)
        #if count >= 2:
        #    break

In [7]:
def get_evt_types(df):
    event_types = []
    for i in range(0,df.shape[0]):
        event_types.append(str(df.iloc[i].name))
    return event_types

In [8]:
def get_date(df):
    if df.shape[0]>0:
        return df.iloc[0][0]
    else:
        return None

In [9]:
def get_loc(df):
    if df.shape[0]>0:
        return df.iloc[0][1]
    else:
        return None

In [10]:
def cut_up_url(url):
    flag = 'meetingid='
    start = url.find(flag)+len(flag) 
    end   = url.find('&')
    substr = url[start:end]
    return substr

In [11]:
def get_id(df):
    if df.shape[0]>0:
        return cut_up_url(df.iloc[0][3])
    else:
        return None

In [12]:
columns2delete = ['club','rb_handicap','rb_personal_best',
                  'rb_season_best','strava_profile']

In [17]:
def flatten_df(df, uniqueID):
    types  = get_evt_types(df)
    date   = get_date(df)
    loc    = get_loc(df)
    m_id   = get_id(df)
    frames = []  # Let's hold all frames
    for i in types:
        temp = i
        df_race = json_normalize(df['races'][i]['results'])
        df_race = df_race.drop(columns2delete, axis=1)
        frames.append(df_race)
     
    # finish up, add meta data
    #print('flat_size = ', len(frames), date)
    if len(frames) > 0:
        total_frame = pd.concat(frames) 
        total_frame['race_data'] = date
        total_frame['race_location'] = loc
        total_frame['meeting_id'] = m_id
        total_frame['uniqueID'] = uniqueID
        return total_frame
    else:
        return None

In [14]:
columns2keep=['age_group', 'chip_time', 'gun_time', 'name', 'position', 
              'race_title', 'sex', 'terrain']

In [18]:
# Main driver cell
final_df_frames = []
ID = 0
for i in dfs:
    subdf = flatten_df(i,ID)
    final_df_frames.append(subdf)
    ID += 1
print(len(final_df_frames))
final_df = pd.concat(final_df_frames)    

12014


In [19]:
final_df.to_csv('testing.csv',sep=',')